# Flask run.py

In [16]:
# import libraries
import json
import nltk
from nltk import download
#nltk.download(['punkt', 'stopwords', 'wordnet'])
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from flask import Flask
from flask import render_template, request, jsonify
import pandas as pd
import pickle
import plotly
from plotly.graph_objs import Bar, Scatter
import sqlite3
import string

In [17]:
# We had a problem with unpickling the model. Supposedly this will work. 
# That is taking out the function and importing it. It WORKED!
from tokenizer import tokenize

In [18]:
# instantiate flask
app = Flask(__name__)

In [19]:
# load data
con = sqlite3.connect('../data/drp.db')

# Get messages
sql = "SELECT * FROM messages"
df = pd.read_sql_query(sql, con)

# Get scores
sql = "SELECT * FROM scores"
df_scores = pd.read_sql_query(sql, con)

# close the sqlite connection
con.close()

# Rename the first column to category
df_scores.rename(columns = {df_scores.columns[0]: 'category'}, inplace=True)

# Melt the df_scores to make it easy to do a line plot
df_melt = df_scores.melt(id_vars = 'category', 
                         value_vars = ['accuracy', 'precision', 'recall', 'f1'],
                         var_name = 'metric',
                         value_name = 'score')

# load model
model = pickle.load(open("../models/model.pkl", "rb"))

In [20]:
# index webpage displays cool visuals and receives user input text for model
@app.route('/')
@app.route('/index')
def index():
    
    """Does plotting for the web page. There are 3 plots. 2 bar plots and 1 line plot. 
    The line plot is based on the scoring of the model"""
    
    # data for first plot
    genre_counts = df.groupby('genre').count()['message']
    genre_names = list(genre_counts.index)
    
    # Add up the number of messages per category in sorted order for the second plot.
    df_2nd = df.iloc[:, 3:].sum().sort_values().reset_index()
    
    # change column names to Category and Count
    df_2nd.columns = ['category','count']
    
    # Sort df_scores by category alphabetically
    df_scores.sort_values('category', inplace=True)

    # create visuals
    
    graphs = [
        {
            'data': [
                Bar(
                    x = genre_names,
                    y = genre_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Genres',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            }
        },
        {
            'data': [
                Bar(
                    x = df_2nd['category'],
                    y = df_2nd['count'],
                    marker = dict(color='green')
                )
            ],

            'layout': {
                'title': "Numbers of Messages Per Message Category"
            }
        },
                {
            'data': [
                Scatter(
                    x = df_scores['category'],
                    y = df_scores['accuracy'],
                    name = 'Accuracy'
                ),
                Scatter(
                    x = df_scores['category'],
                    y = df_scores['precision'],
                    name = 'Precision'
                ),
                Scatter(
                    x = df_scores['category'],
                    y = df_scores['recall'],
                    name = 'Recall'
                ),
                Scatter(
                    x = df_scores['category'],
                    y = df_scores['f1'],
                    name = 'F1'
                )
            ],

            'layout': {
                'title': 'Evaluation Metrics For Each Category',
                'xaxis': {
                    'title':"",
                },
                'yaxis': {
                    'title': "Score",
                }
            }
        }

    ]

    # encode plotly graphs in JSON
    ids = ["graph-{}".format(i) for i, _ in enumerate(graphs)]
    graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)
    
    # render web page with plotly graphs
    return render_template('master.html', ids=ids, graphJSON=graphJSON)


In [21]:
# web page that handles user query and displays model results
@app.route('/go')
def go():
    # save user input in query
    query = request.args.get('query', '') 

    # use model to predict classification for query
    classification_labels = model.predict([query])[0]
    classification_results = dict(zip(df.columns[4:], classification_labels))

    # This will render the go.html Please see that file. 
    return render_template(
        'go.html',
        query=query,
        classification_result=classification_results
    )


In [22]:
def main():
    app.run(host='0.0.0.0', port=3001, debug=True)
    

In [ ]:
if __name__ == '__main__':
    main()